# [DM-16399] Backfill ap_verify CI jobs into SQuaSH 
At the time  of this writing `ap_verify` pipeline in CI is not running `dispatch_verify`. In order to have test data  for defining the mapping between  `lsst.verify` and InfluxDB this notebook will reconstruct `ap_verify` verification jobs and send them to SQuaSH. The SQuaSH API is connected to InfluxDB so the results will appear in Chronograf as well.

Note that in order to restore the original timestamp for the jobs, the SQuaSH API must be running with `SQUASH_ETL_MODE=True`.

In [ ]:
# Use the SQuaSH demo instance, never use production for this type of work.
SQUASH_API_URL = "https://squash-restful-api-demo.lsst.codes"

## Grab ap_verify jobs from Jenkins and send to SQuaSH

In [ ]:
# GitHub auth tokento access Jenkins (https://github.com/settings/tokens)
token = ""

In [ ]:
from dateutil.parser import parse

# Environment variables needed to reconstruct the verification job
env_name = "jenkins"
ci_name = "ap_verify"
ci_dataset = "CI-HiTS2015"

# List of `ap_verify` runs from the Jenkins API 
ap_verify_url = "https://ci.lsst.codes/job/scipipe/job/ap_verify/api/json"
ap_verify = requests.get(ap_verify_url, auth=('afausti', token)).json()

# Loop over the CI runs
for build in ap_verify['builds']:
    
    ci_id = build['number']
    ci_url = build['url']
    
    print("Sending CI job {}".format(ci_id))
    
    # Get an access token for the SQuaSH API, this is done here to avoid
    # token to expire
    r = requests.post(SQUASH_API_URL + "/auth", json='testuser')
    headers = dict()
    headers['Authorization'] = 'JWT ' + r.json()['access_token']

    
    # Retrieve the original timestamp for this run
    build_timestamp_url = "https://ci.lsst.codes/job/scipipe/job/ap_verify/{}/buildTimestamp".format(ci_id)
    build_timestamp = requests.get(build_timestamp_url, auth=('afausti', token)).text
    
    date = parse(build_timestamp).strftime("%Y-%m-%dT%H:%M:%SZ")

    # dataIds processed by ap_verify (I was not able to retrieve these from the Jenkins API)

    data_ids=[{'visit': 411371, 'ccdnum': 56, 'filter': 'g'}, 
              {'visit': 411371, 'ccdnum': 60, 'filter': 'g'},
              {'visit': 411420, 'ccdnum': 10, 'filter': 'g'},
              {'visit': 411420, 'ccdnum': 5,  'filter': 'g'},
              {'visit': 419802, 'ccdnum': 10, 'filter': 'g'},
              {'visit': 419802, 'ccdnum': 5,  'filter': 'g'}]

    for data_id in data_ids:

        # Retrieve the verification measurements 
        artifact_url = ci_url + "artifact/ci-hits2015/ap_verify/{}/ap_verify.visit={}%20ccdnum={}%20filter={}.verify.json".format(ci_dataset, data_id['visit'], data_id['ccdnum'], data_id['filter'])
        print(artifact_url)
        
        r = requests.get(artifact_url, auth=('afausti', token))

        data = r.json()
        
        # Reconstruct job environment metadata
        env = {'ci_id': ci_id, 'ci_name': ci_name, 'ci_dataset': ci_dataset, 'date': date, 'env_name': env_name}

        data['meta']['env'] = env

        # Package information is not reconstructed here.
        data['meta']['packages'] = "" 

        # Reconstruct other job level metadata
        data['meta']['visit'] = data_id['visit']
        data['meta']['ccdnum'] = data_id['ccdnum']
        data['meta']['filter'] = data_id['filter']

        r = requests.post(SQUASH_API_URL + "/job", json=data, headers=headers)
        r.json()

